<a href="https://www.kaggle.com/code/amirulmahmud/classification-of-rock-mine-with-knn?scriptVersionId=124936047" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Load The Data**

Sonar (sound navigation ranging) is a technique that uses sound propagation to navigate, communicate with or detect objects on or under the surface of the water.
The dataset contains the response metrics for 60 separate sonar frequencies sent out against a known mine field and known rocks. These frequencies are then labeled with the known object they were beaming the sound at (either a rock or a mine).
The objective is to create a machine learning model that is capable of classifying a rock or a mine based on the response of the 60 separate sonar frequencies.

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/rock-or-mine-classification/ROCK_OR_MINE.csv', header=None)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df = df.rename(columns={60: 'Label'})

In [ ]:
df.columns

# Data Cleaning

Before data is used, let's check if there is any missing values in the data.

In [ ]:
df.isna().sum()

In [ ]:
df.isna().sum().sum()

Fortunately, the data is complete and does not have any missing values.

# Exploratory Data Analysis

Create a heatmap to visualize the correlation between the difference frequency responses.

In [ ]:
df['Target'] = df['Label'].map({"R":0, "M":1})

In [ ]:
plt.figure(figsize=(8,6),dpi=200)
sns.heatmap(df.corr(), cmap='coolwarm')

Find the top 5 correlated frequencies with target.

In [ ]:
np.abs(df.corr()['Target']).sort_values().tail(6)

In [ ]:
sns.boxplot(x=df['Label'],y=df[10])

In [ ]:
sns.boxplot(x=df['Label'],y=df[11])

In [ ]:
sns.boxplot(x=df['Label'],y=df[48])

Check the statistic summaries

In [ ]:
df.describe().transpose()

# Split The Data

The approach here will use Cross Validation on 80% of the dataset, and then judge the results on a final test set of 20% to evaluate the model.

In [ ]:
df.columns

In [ ]:
X = df.drop(['Label','Target'],axis=1)
y = df['Label']

In [ ]:
X.columns

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Check the balance of the label data in training set.

In [ ]:
sns.countplot(x=y_train)

In [ ]:
y_train.value_counts()

As we see here, the label class of training data is almost balance. So, the data is ready to use in the next steps.

# Modelling with KNN

Create a PipeLine that contains both a StandardScaler and a KNN model

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
scaler = StandardScaler()

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipe = Pipeline([('scaler',scaler), ('knn',knn)])

Perform a grid-search with the pipeline to test various parameters and report back the best performing parameters.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = ({'knn__n_neighbors': list(range(1,30)),
              'knn__weights': ['uniform', 'distance'],
              'knn__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']})

In [ ]:
grid = GridSearchCV(estimator=pipe,param_grid=parameters,scoring='accuracy',cv=5)

In [ ]:
grid.fit(X_train,y_train)

Find the best estimator and best parameters

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_estimator_.get_params()

Cross Validation Results

In [ ]:
cv_results = pd.DataFrame(grid.cv_results_)

In [ ]:
cv_results.info()

In [ ]:
cv_score = cv_results.groupby('param_knn__n_neighbors').agg('mean')['mean_test_score']

In [ ]:
cv_score

In [ ]:
plt.figure(figsize=(5,4),dpi=150)
plt.plot(range(1,30),cv_score,'o-')
plt.xlabel('K Value')
plt.ylabel('Accuracy')

# Final Model Evaluation

In [ ]:
y_pred = grid.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
accuracy_score(y_test,y_pred)

# CONCLUSION

1. The best parameters of KNN estimator in this model are n_neighbors = 1, weights = 'uniform', and algorithm = 'auto'.
2. The model performs quite well in predicting the unseen data (X_test), with accuracy of 90.48%